In [1]:
!pip install streamlit pyngrok fer librosa soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.1/891.1 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1

In [1]:
%%writefile app.py
import streamlit as st
import pandas as pd
import librosa
import soundfile as sf
from fer.fer import FER
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import os
from PIL import Image

# Set page config
st.set_page_config(page_title="Multimodal Emotion-Aware Platform", layout="centered")

# Initialize emotion detector WITHOUT mtcnn (to avoid crash)
emotion_detector = FER()

# Initialize CSV
csv_path = "/content/multimodal_emotion_dataset.csv"
if not os.path.exists(csv_path):
    pd.DataFrame(columns=["Timestamp", "Text", "Audio_Emotion", "Facial_Emotion"]).to_csv(csv_path, index=False)

# Title
st.title("🧠 Multimodal Emotion-Aware Platform")
st.markdown("This app analyzes **Text**, **Voice**, and **Facial Expressions** to detect trainee's emotional state.")

# TEXT Input
st.header("1. Text Input")
text_input = st.text_area("Enter text here:")
text_emotion = "Neutral"
text_emotion_keywords = {
    "happy": "Happy",
    "joy": "Happy",
    "delight": "Happy",
    "sad": "Sad",
    "cry": "Sad",
    "depressed": "Sad",
    "angry": "Angry",
    "mad": "Angry",
    "furious": "Angry",
    "fear": "Fear",
    "scared": "Fear",
    "afraid": "Fear",
    "surprised": "Surprise",
    "shock": "Surprise",
    "disgust": "Disgust"
}

if text_input:
    for keyword, emotion in text_emotion_keywords.items():
        if keyword in text_input.lower():
            text_emotion = emotion
            break
    st.success(f"Detected emotion from text: **{text_emotion}**")

# AUDIO Input
st.header("2. Audio Upload")
audio_file = st.file_uploader("Upload audio file", type=["wav", "mp3"])
audio_emotion = "Unknown"

if audio_file:
    st.audio(audio_file)
    y, sr = librosa.load(audio_file, duration=5)
    rms = np.mean(librosa.feature.rms(y=y))
    pitch = np.mean(librosa.yin(y, fmin=50, fmax=300))

    if pitch > 200 and rms > 0.05:
        audio_emotion = "Excited"
    elif pitch < 130 and rms < 0.02:
        audio_emotion = "Calm"
    elif rms > 0.07:
        audio_emotion = "Angry"
    else:
        audio_emotion = "Neutral"

    st.success(f"Detected emotion from audio: **{audio_emotion}**")

# FACIAL Emotion Detection
st.header("3. Facial Emotion from Image")
img_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
facial_emotion = "Unknown"

if img_file:
    img = Image.open(img_file)
    st.image(img, caption="Uploaded Image", use_container_width=True)

    result = emotion_detector.detect_emotions(np.array(img))
    if result:
        top_emotion = max(result[0]["emotions"], key=result[0]["emotions"].get)
        facial_emotion = top_emotion.capitalize()
        st.success(f"Detected facial emotion: **{facial_emotion}**")
    else:
        st.warning("No face detected!")

# Save to CSV
if st.button("Save to Log"):
    new_row = {
        "Timestamp": datetime.datetime.now().isoformat(),
        "Text": text_emotion,
        "Audio_Emotion": audio_emotion,
        "Facial_Emotion": facial_emotion
    }
    df = pd.read_csv(csv_path)
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df.to_csv(csv_path, index=False)
    st.success("Logged to multimodal_emotion_dataset.csv")

# Show Emotion Log
if st.checkbox("Show Emotion Log"):
    st.subheader("Emotion Detection History")
    df = pd.read_csv(csv_path)
    st.dataframe(df)

# Plot Emotion Development Graphs
if st.checkbox("Show Trainee's Emotion Development Graphs"):
    df = pd.read_csv(csv_path)
    if df.empty:
        st.warning("No data available to plot.")
    else:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"])

        fig, axes = plt.subplots(3, 1, figsize=(10, 12))
        sns.lineplot(data=df, x="Timestamp", y="Text", ax=axes[0])
        axes[0].set_title("Text Emotion Over Time")
        axes[0].set_ylabel("Text Emotion")

        sns.lineplot(data=df, x="Timestamp", y="Audio_Emotion", ax=axes[1])
        axes[1].set_title("Audio Emotion Over Time")
        axes[1].set_ylabel("Audio Emotion")

        sns.lineplot(data=df, x="Timestamp", y="Facial_Emotion", ax=axes[2])
        axes[2].set_title("Facial Emotion Over Time")
        axes[2].set_ylabel("Facial Emotion")

        plt.tight_layout()
        st.pyplot(fig)

# Footer
st.caption("Built for Nursing VR Empathy Training – by [Leena & Sanjna]")


Overwriting app.py


In [2]:
!pip install streamlit fer==25.10.3 librosa soundfile matplotlib seaborn numpy pillow pandas


In [3]:
!pip install pyngrok


In [27]:
!streamlit run app.py & sleep 5 && curl -s http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.149.52:8501

  Stopping...


In [2]:
!wget -q -c https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip


Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [5]:
!./ngrok authtoken 30fmjusdYDIrGEG4tvkMfv85Kyo_7xdp33nJBY6HUV2PxmveE


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
!./ngrok http 8501 &


In [7]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip


Archive:  ngrok.zip
  inflating: ngrok                   


In [8]:
!./ngrok config add-authtoken 30fmjusdYDIrGEG4tvkMfv85Kyo_7xdp33nJBY6HUV2PxmveE


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
!streamlit run app.py &>/content/logs.txt &


In [10]:
!./ngrok http 8501 &>/dev/null &


In [11]:
import time, requests

time.sleep(5)
url = requests.get("http://localhost:4040/api/tunnels").json()['tunnels'][0]['public_url']
print("🌐 Public URL:", url)


🌐 Public URL: https://6103c40b113c.ngrok-free.app


In [10]:
!streamlit run app.py &>/content/logs.txt &


In [11]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip


Archive:  ngrok.zip
  inflating: ngrok                   


In [12]:
!./ngrok config add-authtoken 30fmjusdYDIrGEG4tvkMfv85Kyo_7xdp33nJBY6HUV2PxmveE


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [13]:
!ls


app.py	logs.txt  ngrok  ngrok-stable-linux-amd64.zip  ngrok.zip  sample_data


In [14]:
!streamlit run app.py &>/content/logs.txt &


In [15]:
!pip install streamlit


In [16]:
!pip install fer

In [17]:
!streamlit run app.py &>/content/logs.txt &


In [18]:
!tail -n 20 /content/logs.txt





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8504
  Network URL: http://172.28.0.12:8504
  External URL: http://34.125.149.52:8504



In [19]:
!./ngrok http 8501 &>/dev/null &


In [20]:
import time, requests

time.sleep(5)
url = requests.get("http://localhost:4040/api/tunnels").json()['tunnels'][0]['public_url']
print("🌍 Public URL:", url)


🌍 Public URL: https://b45d64096b8f.ngrok-free.app


In [21]:
from pyngrok import ngrok
import time
import threading

# Run streamlit in a separate thread
def run_app():
    !streamlit run app.py

thread = threading.Thread(target=run_app)
thread.start()

# Wait a bit for Streamlit to spin up
time.sleep(5)

# Open the correct HTTP tunnel for port 8501
public_url = ngrok.connect(addr=8501, proto="http")
print("Your app is live at:", public_url)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8505
  Network URL: http://172.28.0.12:8505
  External URL: http://34.125.149.52:8505



ERROR:pyngrok.process.ngrok:t=2025-10-23T08:46:54+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-10-23T08:46:54+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-10-23T08:46:54+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [22]:
!pip show fer


Name: fer
Version: 25.10.3
Summary: Facial expression recognition from images
Home-page: https://github.com/justinshenk/fer
Author: Justin Shenk
Author-email: Justin Shenk <shenkjustin@gmail.com>
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: facenet-pytorch, ffmpeg-python, matplotlib, moviepy, opencv-contrib-python, pandas, Pillow, requests, tensorflow, tqdm
Required-by: 
